# Demo 4: Model fitting

In Demo 4, we will be focusing on the basics of fitting actual behavioral data, including the pitfalls of certain common mistakes.
The demo will be split into 3 parts.
Each demo will be accompanied by **guiding questions**, which will help with the learning process as we go.
    
In **Part A**, we will explore how the prior controls the pattern of hand localizations observed in an experiment.

In **Part B**, we will simulate an actual hand localization experiment and fit it using our computational model.

In **Part C**, we will explore how choices in the nature of the computational model affect model fitting performance.


## Demo 4A: Influence of the Prior on patterns of responses

In the current demo, we will explore how the paramters of the Prior distribution control the patterns of behavioral responses observed
in a hand localization experiment. The below plot illlustrates the likelihood distribution (red curves) for three locations, a prior distribution (blue curve),
and the response distributions (green curves). 

In the demo, you can adjust  the measurement noise, and the parameters of the prior distribution (i.e., mean and uncertainty).
We will be focusing on how these parameters affect different aspects of the localization biases, i.e., the differences 
between the likelihood and response distributions.
    
### Guiding Questions
1. How does the level of measurement noise affect the different aspects (offsets, spread) and magnitude of the localization biases?

2. How does the prior mean affect the different aspects (offsets, spread) and magnitude of the localization biases?

3. How does the level of prior uncertainty affect the different aspects (offsets, spread) and magnitude of the localization biases?


In [1]:

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, IntSlider, Checkbox, VBox, HBox
from IPython.display import display

# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120

# ---------------------------------------------------------------------------
# Global axis limits (stay *constant* no matter what the sliders do)
SIGMA_RANGE = (1, 5)    # from sliders

PR_RANGE = (-10, 10)
SIGMA_PR_RANGE = (1, 10)
StimLim = (-10, 10)
STIM_LOCS = (-10, -5, 0, 5, 10)
XLIM = (-20, 20)   # (-17, 17)
YLIM = (-0.002, 0.05)         # room for baseline + PDF peak (0.15)




def show_locs(sigma: float = 1.0, 
                  mu_pr: float = 0.0, 
                  sigma_pr: float = 1.0):

    fig, ax = plt.subplots(figsize=(8, 6))

    stim_locs = (-10, 0, 10)
    # locs2show = (-10, 0, 10)
    

    var_m = sigma**2
    var_pr = sigma_pr**2
    var_post = (var_m**-1+var_pr**-1)**-1
    Wp = var_m**-1/(var_m**-1 + var_pr**-1)
    Wpr = 1-Wp

   
    xs = np.linspace(XLIM[0], XLIM[1], 400)

    for i in range(3):
        mu_L = stim_locs[i]
        
        mu_post = Wp*stim_locs[i] + Wpr*mu_pr
        mu_rd = mu_post
        var_rd = Wp**2*var_m
        norm_post = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - 0)** 2 / var_rd) ).sum()

        pdf_post = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - mu_rd)** 2 / var_rd) )
        pdf_post_norm = pdf_post / norm_post # Scaling the pdf for better visual
        ax.plot(xs, pdf_post_norm, color="green", linewidth=2)

        norm_L = (1 / (np.sqrt(2 * np.pi) * var_m)) * np.exp(-0.5 * ((0-xs)** 2 / var_m) ).sum()

        pdf_L = (1 / (np.sqrt(2 * np.pi) * var_m)) * np.exp(-0.5 * ((mu_L-xs)** 2 / var_m) )
        pdf_L_norm = pdf_L / norm_L # Scaling the pdf for better visual
        ax.plot(xs, pdf_L_norm, color="red", linewidth=2)
    
    norm_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - 0)** 2 / var_pr) ).sum()
    pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - mu_pr)** 2 / var_pr) )
    pdf_pr_norm = pdf_pr / norm_pr # Scaling the pdf for better visual
    ax.plot(xs, pdf_pr_norm, color="blue", linewidth=2)

    # Fixed axes
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.45*XLIM[1],0.9*YLIM[1],'Likelihood Dists.',color="red",fontsize=12)
    ax.text(0.45*XLIM[1],0.83*YLIM[1],'Prior Dist.',color="blue",fontsize=12)
    ax.text(0.45*XLIM[1],0.76*YLIM[1],'Responses Dists.',color="green",fontsize=12)
    
    # Aesthetics

    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()


# ---------------------------------------------------------------------------
# controls
controls = {
    "sigma": FloatSlider(value=3.0, min=SIGMA_RANGE[0], max=SIGMA_RANGE[1], step=0.1,
                          description="Noise σ:", continuous_update=True),
    "mu_pr": FloatSlider(value=5.0, min=PR_RANGE[0], max=PR_RANGE[1], step=0.1,
                       description="Prior:", continuous_update=True,
                       readout_format=".1f"),
    "sigma_pr": FloatSlider(value=3.0, min=SIGMA_PR_RANGE[0], max=SIGMA_PR_RANGE[1], step=0.1,
                          description="Prior σ:", continuous_update=True),

}

interactive_plot = interactive(show_locs, **controls)

# Two‑column layout: sliders left, plot right
ui_left = VBox([controls["sigma"], controls["mu_pr"],controls["sigma_pr"]])

output_area = interactive_plot.children[-1]

display(HBox([ui_left, output_area]))
interactive_plot.update()

## Demo 4B: Influence of the Prior on patterns of responses

In the current demo, we will simulate a hand localization experiment and fit the data using the Bayesian model that we've been learning
about today. 

In this hypothetical experiment, a participant localizes their hand, which is placed at five different locations
across the space from -10 to +10. We can determine how many trials per location we have in the experiment, using the parameter "# of trials".

We also have control over three aspect of our generative model: Measurement noise, prior mean, and prior uncertainty. The model fitting procedure
will then try to recover these parameters, given the trial-by-trial localization.

In the plot, you will see the likelihood distributions (red curves) for three of the locations, a prior distribution (blue curve),
and the response distributions (green curves). The dashed purple line corresponds to a localization on a given trial.

Checking the box "Fit the data" will perform our model-fitting procedure (minmization of the -LL), and the recovered parameters will
be illustrated using distributions: Fitted likelihood (grey curve), fitted prior (orange curve) and fitted response distributions (black curve).

    
### Guiding Questions
1. How well does the model fitting work with the default parameters? Fit the model several times by clicking the checkbox. Does the fit seem consistent?

2. How does the number of trials per location affect the model fitting? Can we accurately recover the underlying parameters with only a few trials per location?
To observe this in real time, you can keep the "fit model" checkbox clicked and slide "# of trials" up and down.

3. Finally, let's see if the fitting ability depends upon the underlying generative model. Explore several combinations of prior uncertainty.
Is the fit better or worse when these values are high? Why might that be the case?





In [2]:

import scipy as sp

# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120

# ---------------------------------------------------------------------------
# Global axis limits (stay *constant* no matter what the sliders do)
SIGMA_RANGE = (1, 5)    # from sliders

PR_RANGE = (-10, 10)
SIGMA_PR_RANGE = (1, 10)
StimLim = (-10, 10)
STIM_LOCS = (-10, -5, 0, 5, 10)
XLIM = (-20, 20)   # (-17, 17)
YLIM = (-0.002, 0.05)         # room for baseline + PDF peak (0.15)


def sim_data(stimLocs, params):
    sigma = params[0]
    sigma_pr = params[1]
    mu_pr = params[2]
    n_samples = params[3]
    var_m = sigma**2
    var_pr = sigma_pr**2
    Wp = var_m**-1/(var_m**-1 + var_pr**-1)
    Wpr = 1-Wp

    rng = np.random.default_rng()

    responses = []
    stimuli = []

    for i in range(len(stimLocs)):
        currStim = stimLocs[i]

        for n in range(n_samples):
            x_obs = rng.normal(loc=currStim, scale=sigma, size=1)
            posterior = Wp*x_obs + Wpr * mu_pr
            responses.append(posterior)
            stimuli.append(currStim)
    data = np.append(responses, stimuli)
    return data

def obj_func(params,data):
    sigma_p = params[0]
    sigma_pr = params[1]
    mu_pr = params[2]
    
    n = len(data)
    responses = data[0:int(n/2)]
    stimuli = data[int(n/2):int(n)]

    w_p = sigma_p**-2/(sigma_p**-2+sigma_pr**-2)
    w_pr = 1 - w_p

    LL_all = []

    mu_post = w_p*stimuli + w_pr*mu_pr
    mu_rd = mu_post
    var_int = (sigma_p**-2+sigma_pr**-2)**-1
    var_rd = w_p**2*sigma_p**2
    sq_diffs = (responses - mu_rd)**2
    n_resp = int(n/2)/2
    LL = n_resp*np.log(2*np.pi) - n_resp*np.log(var_rd)-(1/(2*var_rd))*sq_diffs.sum()
    nLL = -LL.sum()
    return nLL




def show_fit(sigma: float = 1.0, 
                  mu_pr: float = 0.0, 
                  sigma_pr: float = 1.0,
                  n_samples: float = 1.0,
                  fit_data: bool = True):

    fig, ax = plt.subplots(figsize=(8, 6))

    stim_locs = (-10, 0, 10, -5,  5)
    # locs2show = (-10, 0, 10)
    

    var_m = sigma**2
    var_pr = sigma_pr**2
    var_post = (var_m**-1+var_pr**-1)**-1
    Wp = var_m**-1/(var_m**-1 + var_pr**-1)
    Wpr = 1-Wp

   
    xs = np.linspace(XLIM[0], XLIM[1], 400)

    for i in range(3):
        mu_L = stim_locs[i]
        
        mu_post = Wp*stim_locs[i] + Wpr*mu_pr
        mu_rd = mu_post
        var_rd = Wp**2*var_m
        norm_post = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - 0)** 2 / var_rd) ).sum()

        pdf_post = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - mu_rd)** 2 / var_rd) )
        pdf_post_norm = pdf_post / norm_post # Scaling the pdf for better visual
        ax.plot(xs, pdf_post_norm, color="green", linewidth=2)

        norm_L = (1 / (np.sqrt(2 * np.pi) * var_m)) * np.exp(-0.5 * ((0-xs)** 2 / var_m) ).sum()

        pdf_L = (1 / (np.sqrt(2 * np.pi) * var_m)) * np.exp(-0.5 * ((mu_L-xs)** 2 / var_m) )
        pdf_L_norm = pdf_L / norm_L # Scaling the pdf for better visual
        ax.plot(xs, pdf_L_norm, color="red", linewidth=2)
    
    norm_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - 0)** 2 / var_pr) ).sum()
    pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - mu_pr)** 2 / var_pr) )
    pdf_pr_norm = pdf_pr / norm_pr # Scaling the pdf for better visual
    ax.plot(xs, pdf_pr_norm, color="blue", linewidth=2)

    
    data = sim_data(stim_locs,[sigma,sigma_pr,mu_pr,n_samples])

    trials = list(range(n_samples*3))
    data2show = data[trials]



    # nLL = obj_func(data,[1,1,1])

    init_params = [1,1,1]

    p_bounds = [(0.1, 25),(0.1,25),(-40, 40)]
    

    if fit_data:
        fitted = sp.optimize.minimize(obj_func,
                                      init_params,
                                      method = "Nelder-Mead",
                                      args = data, 
                                      bounds = p_bounds, 
                                      tol= 1e-6)
        params_fit = fitted.x

        var_m_fit = params_fit[0]**2
        var_pr_fit = params_fit[1]**2
        mu_pr_fit = params_fit[2]
        
        var_post = (var_m_fit**-1+var_pr_fit**-1)**-1
        Wp = var_m_fit**-1/(var_m_fit**-1 + var_pr_fit**-1)
        Wpr = 1-Wp
    
  
        xs = np.linspace(XLIM[0], XLIM[1], 400)
        for i in range(3):

            mu_L = stim_locs[i]
            mu_post = Wp*stim_locs[i] + Wpr*mu_pr_fit
            mu_rd = mu_post
            var_rd = Wp**2*var_m_fit
            norm_post = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - 0)** 2 / var_rd) ).sum()
    
            pdf_post = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - mu_rd)** 2 / var_rd) )
            pdf_post_norm = pdf_post / norm_post # Scaling the pdf for better visual
            ax.plot(xs, pdf_post_norm, color="black", linewidth=2)


            norm_L = (1 / (np.sqrt(2 * np.pi) * var_m_fit)) * np.exp(-0.5 * ((0-xs)** 2 / var_m_fit) ).sum()
    
            pdf_L = (1 / (np.sqrt(2 * np.pi) * var_m_fit)) * np.exp(-0.5 * ((mu_L-xs)** 2 / var_m_fit) )
            pdf_L_norm = pdf_L / norm_L # Scaling the pdf for better visual
            ax.plot(xs, pdf_L_norm, color="grey", linewidth=2)


        
        norm_pr = (1 / (np.sqrt(2 * np.pi) * var_pr_fit)) * np.exp(-0.5 * ((xs - 0)** 2 / var_pr_fit) ).sum()
        pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr_fit)) * np.exp(-0.5 * ((xs - mu_pr_fit)** 2 / var_pr_fit) )
        pdf_pr_norm = pdf_pr / norm_pr # Scaling the pdf for better visual
        ax.plot(xs, pdf_pr_norm, color="orange", linewidth=2)
    else:
        for r in list(range(n_samples*3)):
            ax.axvline(data2show[r], color="purple", linestyle="--")

    # Fixed axes
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.60*XLIM[1],0.9*YLIM[1],'ACTUAL',color="black",fontsize=12)
    ax.text(0.55*XLIM[1],0.83*YLIM[1],'Likelihood',color="red",fontsize=12)
    ax.text(0.55*XLIM[1],0.76*YLIM[1],'Prior',color="blue",fontsize=12)
    ax.text(0.55*XLIM[1],0.69*YLIM[1],'Responses',color="green",fontsize=12)
    
    ax.text(-13,0.9*YLIM[1],'FITTED',color="black",fontsize=12)
    ax.text(-15,0.83*YLIM[1],'Likelihood',color="grey",fontsize=12)
    ax.text(-15,0.76*YLIM[1],'Prior',color="orange",fontsize=12)
    ax.text(-15,0.69*YLIM[1],'Responses',color="black",fontsize=12)
    
    # Aesthetics

    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()


# ---------------------------------------------------------------------------
# controls
controls = {
    "sigma": FloatSlider(value=3.0, min=SIGMA_RANGE[0], max=SIGMA_RANGE[1], step=0.1,
                          description="Noise σ:", continuous_update=True),
    "mu_pr": FloatSlider(value=5.0, min=PR_RANGE[0], max=PR_RANGE[1], step=0.1,
                       description="Prior:", continuous_update=True,
                       readout_format=".1f"),
    "sigma_pr": FloatSlider(value=3.0, min=SIGMA_PR_RANGE[0], max=SIGMA_PR_RANGE[1], step=0.1,
                          description="Prior σ:", continuous_update=True),
    "n_samples": IntSlider(value=10, min=2, max=100, step=1,
                           description="# of trials:", continuous_update=True),
    "fit_data": Checkbox(value=False, description="Fit the data"),

}

interactive_plot = interactive(show_fit, **controls)

# Two‑column layout: sliders left, plot right
ui_left = VBox([controls["sigma"], controls["mu_pr"],controls["sigma_pr"],controls["n_samples"],
                controls["fit_data"]])

output_area = interactive_plot.children[-1]

display(HBox([ui_left, output_area]))
interactive_plot.update()

## Demo 4C: How different mistakes affect fitting performance 

The current demo will explore how different experimental/modelling mistakes will affect your ability to fit the data you've collected.
The demo is almost exaclty the same as in 4B, with the following additional options:

The number of hand locations in the experiment: "# of locs"

Modelling a prior or not: "Model a prior?"

Confusing the Response and Prior distributions: "RD = PD"

Additional sources of noise (e.g., response noise) that go unmodelled: "Unmodelled noise"

    
### Guiding Questions
1. How does the number of measured locations affect the model fit? Is there a relationship between number of locations and the number of trials per location?

2. What happens if we do not include a prior in computational model when there is indeed one in the generative model of our data?

3. What effect does confusing the posterior and response distribution have on the recovered parameters? Why?

4.  What happens if there are additional sources of noise in the generative model of our data that go unmodelled? Such as
added noise when making a movement towards the hand.



In [3]:

import scipy as sp

# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120

# ---------------------------------------------------------------------------
# Global axis limits (stay *constant* no matter what the sliders do)
SIGMA_RANGE = (1, 5)    # from sliders

PR_RANGE = (-10, 10)
SIGMA_PR_RANGE = (1, 10)
StimLim = (-10, 10)
STIM_LOCS = (-10, -5, 0, 5, 10)
XLIM = (-20, 20)   # (-17, 17)
YLIM = (-0.002, 0.05)         # room for baseline + PDF peak (0.15)


def sim_data2(stimLocs, params):
    sigma = params[0]
    sigma_pr = params[1]
    mu_pr = params[2]
    n_samples = params[3]
    var_m = sigma**2
    var_pr = sigma_pr**2
    Wp = var_m**-1/(var_m**-1 + var_pr**-1)
    Wpr = 1-Wp

    rng = np.random.default_rng()

    responses = []
    stimuli = []

    for i in range(len(stimLocs)):
        currStim = stimLocs[i]

        for n in range(n_samples):
            x_obs = rng.normal(loc=currStim, scale=sigma, size=1)
            if noise:
                sigma_noise = 2
                x_obs = x_obs + rng.normal(loc=0, scale=sigma_noise, size=1)
            posterior = Wp*x_obs + Wpr * mu_pr
            responses.append(posterior)
            stimuli.append(currStim)
    data = np.append(responses, stimuli)
    return data


    
def obj_func2(params,data):
    sigma_p = params[0]
    sigma_pr = params[1]
    mu_pr = params[2]
    
    n = len(data)
    responses = data[0:int(n/2)]
    stimuli = data[int(n/2):int(n)]

    w_p = sigma_p**-2/(sigma_p**-2+sigma_pr**-2)
    w_pr = 1 - w_p

    LL_all = []

    mu_post = w_p*stimuli + w_pr*mu_pr
    mu_rd = mu_post
    var_int = (sigma_p**-2+sigma_pr**-2)**-1
    if pd_mistake:
        var_rd = w_p**2*sigma_p**2+w_pr**2*sigma_pr**2
    else:
        var_rd = w_p**2*sigma_p**2
    sq_diffs = (responses - mu_rd)**2
    n_resp = int(n/2)/2
    LL = n_resp*np.log(2*np.pi) - n_resp*np.log(var_rd)-(1/(2*var_rd))*sq_diffs.sum()
    nLL = -LL.sum()
    return nLL



def show_fit2(sigma: float = 1.0, 
                  mu_pr: float = 0.0, 
                  sigma_pr: float = 1.0,
                  n_samples: float = 1.0,
                  n_locs: float = 1.0,
                  fit_data: bool = True,
                  fit_prior: bool = True,
                  rd_pd: bool = True,
                  other_noise: bool = True):

    global pd_mistake
    global noise
    
    if rd_pd:
        pd_mistake = 1
    else:
        pd_mistake = 0
   
    if other_noise:
        noise = 1
    else:
        noise = 0 
    
    fig, ax = plt.subplots(figsize=(8, 6))

    if n_locs>2:
        stim_locs = range(-10,10,int(20/n_locs))
        locs2show = (-10, 0, 10)

    else:
        stim_locs = (-10,0,10)
        sim_locs = stim_locs[len(range(n_locs))]
        locs2show = stim_locs



    
    n2show = n_locs

    
    if n2show > 3:
        n2show = 3

    var_m = sigma**2
    var_pr = sigma_pr**2
    var_post = (var_m**-1+var_pr**-1)**-1
    Wp = var_m**-1/(var_m**-1 + var_pr**-1)
    Wpr = 1-Wp

   
    xs = np.linspace(XLIM[0], XLIM[1], 400)

    for i in range(n2show):
        mu_L = locs2show[i]
        
        mu_post = Wp*locs2show[i] + Wpr*mu_pr
        mu_rd = mu_post
        var_rd = Wp**2*var_m
        norm_post = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - 0)** 2 / var_rd) ).sum()

        pdf_post = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - mu_rd)** 2 / var_rd) )
        pdf_post_norm = pdf_post / norm_post # Scaling the pdf for better visual
        ax.plot(xs, pdf_post_norm, color="green", linewidth=2)

        norm_L = (1 / (np.sqrt(2 * np.pi) * var_m)) * np.exp(-0.5 * ((0-xs)** 2 / var_m) ).sum()

        pdf_L = (1 / (np.sqrt(2 * np.pi) * var_m)) * np.exp(-0.5 * ((mu_L-xs)** 2 / var_m) )
        pdf_L_norm = pdf_L / norm_L # Scaling the pdf for better visual
        ax.plot(xs, pdf_L_norm, color="red", linewidth=2)
    
    norm_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - 0)** 2 / var_pr) ).sum()
    pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - mu_pr)** 2 / var_pr) )
    pdf_pr_norm = pdf_pr / norm_pr # Scaling the pdf for better visual
    ax.plot(xs, pdf_pr_norm, color="blue", linewidth=2)

    
    data = sim_data2(stim_locs,[sigma,sigma_pr,mu_pr,n_samples])

    trials = list(range(n_samples*3))
    data2show = data[trials]



    # nLL = obj_func(data,[1,1,1])
    if fit_prior:
        init_params = [1,1,1]

        p_bounds = [(0.1, 25),(0.1,25),(-40, 40)]
    else:
        init_params = [1,100,1]

        p_bounds = [(0.1, 25),(100,100),(-40, 40)]

    
    fitted = sp.optimize.minimize(obj_func2,
                                  init_params,
                                  method = "Nelder-Mead",
                                  args = data, 
                                  bounds = p_bounds, 
                                  tol= 1e-6)
    params_fit = fitted.x

    var_m_fit = params_fit[0]**2
    var_pr_fit = params_fit[1]**2
    mu_pr_fit = params_fit[2]
    
    var_post = (var_m_fit**-1+var_pr_fit**-1)**-1
    Wp = var_m_fit**-1/(var_m_fit**-1 + var_pr_fit**-1)
    Wpr = 1-Wp


    xs = np.linspace(XLIM[0], XLIM[1], 400)
    for i in range(n2show):
        mu_L = locs2show[i]
        mu_post = Wp*locs2show[i] + Wpr*mu_pr_fit
        mu_rd = mu_post
  
        var_rd = Wp**2*var_m_fit
        norm_post = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - 0)** 2 / var_rd) ).sum()

        pdf_post = (1 / (np.sqrt(2 * np.pi) * var_rd)) * np.exp(-0.5 * ((xs - mu_rd)** 2 / var_rd) )
        pdf_post_norm = pdf_post / norm_post # Scaling the pdf for better visual
        ax.plot(xs, pdf_post_norm, color="purple", linewidth=2)


        norm_L = (1 / (np.sqrt(2 * np.pi) * var_m_fit)) * np.exp(-0.5 * ((0-xs)** 2 / var_m_fit) ).sum()

        pdf_L = (1 / (np.sqrt(2 * np.pi) * var_m_fit)) * np.exp(-0.5 * ((mu_L-xs)** 2 / var_m_fit) )
        pdf_L_norm = pdf_L / norm_L # Scaling the pdf for better visual
        ax.plot(xs, pdf_L_norm, color="grey", linewidth=2)


    if fit_prior:
        norm_pr = (1 / (np.sqrt(2 * np.pi) * var_pr_fit)) * np.exp(-0.5 * ((xs - 0)** 2 / var_pr_fit) ).sum()
        pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr_fit)) * np.exp(-0.5 * ((xs - mu_pr_fit)** 2 / var_pr_fit) )
        pdf_pr_norm = pdf_pr / norm_pr # Scaling the pdf for better visual
        ax.plot(xs, pdf_pr_norm, color="orange", linewidth=2)


    # Fixed axes
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.60*XLIM[1],0.9*YLIM[1],'ACTUAL',color="black",fontsize=12)
    ax.text(0.55*XLIM[1],0.83*YLIM[1],'Likelihood',color="red",fontsize=12)
    ax.text(0.55*XLIM[1],0.76*YLIM[1],'Prior',color="blue",fontsize=12)
    ax.text(0.55*XLIM[1],0.69*YLIM[1],'Responses',color="green",fontsize=12)
    
    ax.text(-13,0.9*YLIM[1],'FITTED',color="black",fontsize=12)
    ax.text(-15,0.83*YLIM[1],'Likelihood',color="grey",fontsize=12)
    ax.text(-15,0.76*YLIM[1],'Prior',color="orange",fontsize=12)
    ax.text(-15,0.69*YLIM[1],'Responses',color="black",fontsize=12)
    # Aesthetics

    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()


# ---------------------------------------------------------------------------
# controls
controls = {
    "sigma": FloatSlider(value=3.0, min=SIGMA_RANGE[0], max=SIGMA_RANGE[1], step=0.1,
                          description="Noise σ:", continuous_update=True),
    "mu_pr": FloatSlider(value=5.0, min=PR_RANGE[0], max=PR_RANGE[1], step=0.1,
                       description="Prior:", continuous_update=True,
                       readout_format=".1f"),
    "sigma_pr": FloatSlider(value=3.0, min=SIGMA_PR_RANGE[0], max=SIGMA_PR_RANGE[1], step=0.1,
                          description="Prior σ:", continuous_update=True),
    "n_samples": IntSlider(value=10, min=2, max=100, step=1,
                           description="# of trials:", continuous_update=True),
    "n_locs": IntSlider(value=5, min=1, max=20, step=1,
                           description="# of locs:", continuous_update=True),
    "fit_prior": Checkbox(value=True, description="Model a prior?"),
    "rd_pd": Checkbox(value=False, description="RD = PD?"),
    "other_noise": Checkbox(value=False, description="Unmodelled Noise"),

}

interactive_plot = interactive(show_fit2, **controls)

# Two‑column layout: sliders left, plot right
ui_left = VBox([controls["sigma"], controls["mu_pr"],controls["sigma_pr"],controls["n_samples"],controls["n_locs"],
                controls["fit_prior"],controls["rd_pd"],controls["other_noise"]])

output_area = interactive_plot.children[-1]

display(HBox([ui_left, output_area]))
interactive_plot.update()